In [1]:
import cv2 as cv
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics
from matplotlib import pyplot as plt
import time
import sys
import multiprocessing
import os
import rospy
from sensor_msgs.msg import Image
from stereo_msgs.msg import DisparityImage
from sensor_msgs.msg import PointCloud2
from nav_msgs.msg import OccupancyGrid as Map
import message_filters
import cv_bridge
import tf
import math
from std_msgs.msg import String

In [2]:



def transform(img, angle):
    
# Once a color image has been read, it passes through the 
# transform function. The transform function has two parts:
# pre-processing also known as pp which filters the image based
# of certain values in the hue, saturation, and value (hsv) 
# colorspace and the dbscan which clusters the processed image
# and classifies the clusters as either a buoy, a green buoy,
# a red buoy, or noise. No changes are needed.

    global debug
    start_time = time.time()
    bridge = cv_bridge.CvBridge()
    
    width = img.height
    height = img.width
    img = bridge.imgmsg_to_cv2(img, desired_encoding="passthrough")
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    
    
    
    
    
    
    overlay = False
    image = pp(img,width,height)
    img2 = dbscan(image, img, overlay,angle)
    if debug:
        print("Transform: ", time.time()-start_time)
    return img2



def pp(image, width, height):
    
# The first function in transform is pp or preprocessing. The 
# first line truncates the lower half of the image. The reason
# this is because the plantoons from the wamv are always 
# blocking the camera vision and become noisy data. The
# function changes the image to the HSV colorspace and 
# filters out the image. No changes are needed. It returns 
# the filtered image in the BGR colorspace.The input is the
# color image.

    global debug
    start_time = time.time()
    image = np.array(image[0:width][0:int(height*0.65)])
    #image = cv.GaussianBlur(image, (3,3), sigmaX=0.25, sigmaY=0.25)
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_and(image[:,:,1] > 5,np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10))))] = [0,0,0]
    image = cv.cvtColor(image,cv.COLOR_HSV2BGR)
    if debug:
        print("Preprocessing: ", time.time()-start_time)
    return(image)



def dbscan(image, original, overlay, angle):
    
# The second function in transform is dbscan. The inputs are 
# the filtered colored image and the original color image
# read from the imread statement. The output is the original
# image.This is the end of this classification algorithm.

###########################################################

    global debug
    start_time = time.time()
# Part 1: Thresholding the image
##########################################################
# The reason we threshold the image before applying DBSCAN to 
# the points is because DBSCAN is a computational demanding 
# function. This part takes the filtered image, makes a 
# copy of it, and converts it into a grayscale image. The gray-
# scale image is thresholded at a value of 10. 0 is black and
# 255 is white. This takes the ~400,000 matrix points and 
# leaves only ~10,000 nonzero matrix points. 

    img = image.copy()
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ret,thresh1 = cv.threshold(img,10,255,cv.THRESH_BINARY)

# After thresholding the image, we pull out those nonzero
# points and assign it to the nonzero array variable. This
# array has dimensions of n rows and 2 columns. The first 
# column is the y coordinate and the second column is the x
# coordinate. These two arrays would be combined by 
# np.column_stack which merges the two arrays at their columns.

    nonzero = np.nonzero(thresh1)

    yp = np.array(nonzero[0])
    xp = np.array(nonzero[1])


    X=np.column_stack((xp,yp))
#########################################################

# Creating the BDSCAN object and Sorting Array with Labels
#########################################################    
# After thresholding the filtered image and creating an
# array of nonzero coordinates, this algorithm applies a
# clustering algorithm that filters the image based off the
# array X. If X is empty, this function ends and output is 
# the original input. The first line creates the DBSCAN
# object with an epsilon of 3 and minimum samples of 20. 
# Eps refers to the size of the neighborhood and the 
# min_samples refers to the minimum number of samples required
# for that neighborhood to be considered a cluster. The user
# is free to change the two values for eps and min_samples. The
# .fit(X) applies the matrix points to the DBSCAN object.
# One of the parameters of the db scan object are the labels.
# This is invoked by the line labels = db.labels. This creates
# an array of labels that assigns a number to the image index.
# n clusters have labels that start at 0 and end with n-1.For 
# example, if an image has four clusters, the labels would
# include 0,1,2, and 3. -1 is dedicated for noise. 

    if (len(X) > 0):
        db = DBSCAN(eps=3, min_samples=10).fit(X)
        labels = db.labels_

# The array variable combines the x and y coordinates and the 
# labels in a single array aligned by column. The array is
# sorted with the sort variable with the values in the third
# column or the labels. For example, an image with four
# clusters and noise would be labeled in order from (-1,0,1,2,3).
# The while loop iterates from the beginning of the sorted loop
# and deletes every instance of -1 or until the list is empty.
# This while loop is needed because the noise data is
# irrelevant to the cluster algorithm and the user does not 
# know how much noise there is in the image. The image could 
# have no noise or it could all be noise. The output of this
# section is an array with coordinates sorted together in 
# clusters.

        array = zip(xp,yp,labels)
        sort = sorted(list(array), key=lambda x: x[2])

        x = 0
        while (len(sort) > 0 and sort[x][2] == -1):
            del sort[x]
#########################################################

# Iterating through and classifying each cluster
#########################################################
# After iterating through sort, the program is either left with
# an empty array or an array of coordinates sorted and grouped
# by each cluster. The user needs the number of clusters. The 
# next three lines in the big if block does this. unique_labels
# sorts the set of labels and removes -1 if present. The next 
# two lines creates a copy of the sort array and removes the 
# labels column from the array.

        if (len(sort) > 0):
            unique_labels = sorted(set(labels))
            if (unique_labels[0] == -1):
                unique_labels.remove(-1)
            points = sort
            points = np.delete(points,2,1)

# The program assigns three variables. The first one i iterates
# through sort and compare its label value to x for each 
# iteration until the very end of the sort array. The second
# x refers to the cluster that the program is currently class-
# ifying. It starts with 0 and continues classifying until it 
# has classified all clusters. The third variable is l and this
# array becomes all the points within a cluster. Every time the 
# variable i iterates through sort and is equal to the value of
# the current cluster x, the coordinates are appended to the 
# array l. Since this array is sorted, the first time the x 
# variable equals a different value- the cluster has been 
# accounted for. This array of l contains all the points within
# the cluster. This array and the filtered image gets passed
# through the color function. The result of the color function
# gets passed through the rectangle function with the array,
# filtered image, and original image. The x variable is 
# incremented for the next cluster and the l array is set equal 
# to zero. Once the while loop has iterated through all 
# clusters, the result of the rectangle function is set to the 
# original image and the original image is returned to the 
# transform function which is returned to the read function 
# which writes the image.

            i = 0
            x = 0
            l = []
            while (x < len(unique_labels)):
                if (i < len(sort) and sort[i][2] == x):
                    l.append(points[i])
                    i+=1
                else:
                    x+=1
                    clr = color_hsv(np.array(l),image)
                    tb = rectangle(np.array(l),clr,image,original, overlay,angle)
                    l = []
            original = tb
    if debug:
        print("DBSCAN: ", time.time()-start_time)
    return original



def color(mat,image):
    
    ############################################################

# For the color function, the points of the cluster and the 
# filtered image are passed through. In this function, the 
# average red, green, and blue pixel values are calculated
# from the image inside of the cluster. If the average value
# fits within a certain range, the string 'red' or 'green' is 
# returned. Otherwise, the function returns the string 'none'
# The user is free to change the range of the BGR values that
# determine whether the buoy is red or green.

    sumred = 0
    sumgrn = 0
    sumblu = 0
    for x in range(len(mat)):
        sumred += image [mat[x][1]] [mat[x][0]] [2]
        sumgrn += image [mat[x][1]] [mat[x][0]] [1]
        sumblu += image [mat[x][1]] [mat[x][0]] [0]
    if ((sumred/len(mat)) > 80 and (sumred/len(mat)) < 180 and (sumgrn/len(mat)) > 30 and (sumgrn/len(mat)) < 115  and (sumblu/len(mat)) > 30 and (sumblu/len(mat)) < 110):
        return 'red'
    if ((sumred/len(mat)) > 30 and (sumred/len(mat)) < 80 and (sumgrn/len(mat)) > 60 and (sumgrn/len(mat)) < 180  and (sumblu/len(mat)) > 50 and (sumblu/len(mat)) < 130):
        return 'green'
    else:
        return 'none'

def color_hsv(mat, image):
    
# The rectangle function's parameters are the array of coordinates
# of the cluster, the color of the cluster, the filtered image,
# and the original image. The boundingRect function applies
# a non-rotated rectangle to the image points and returns four
# values in an array, the x and y coordinates of the leftmost
# corner and the width and height. These values are converted
# to ints. The font is set to Hershey Simple for the text and
# the tag is set to zero. The tag is useful in the classifica-
# tion of the buoy. The first classification of the buoy is 
# the proportion of the width and height. If the height is
# between 1.5 times the width and 3.5 times the width, the
# cluster is considered a buoy. If the cluster is given a 
# 'red' or 'green' color, it would be classified as a 'red buoy'
# or 'green buoy' respectively. Otherwise, it would simply be
# classified as a buoy. If any of these three options occur, 
# the tag is set to 1. If the height is not in that range
# mentioned above and the color is 'none', the cluster is
# classified as noise. The last function rectangle makes a 
# rectangle over the original image. The original image is sent
# back to the DBSCAN function.

    global debug
    start_time = time.time()
    image = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    sumhue = 0
    sumsat = 0
    sumval = 0
    mat_add = 0
    for x in range(len(mat)):
        if(image [mat[x][1]] [mat[x][0]] [2] > 5):
            mat_add = mat_add+1
            sumval += image [mat[x][1]] [mat[x][0]] [2]
            sumsat += image [mat[x][1]] [mat[x][0]] [1]
            sumhue += image [mat[x][1]] [mat[x][0]] [0]
    if mat_add > 0:
        sat = sumsat/mat_add
        val = sumval/mat_add
        hue = sumhue/mat_add
        #return (hue, sat, val)
        if (sat > 5 and val > 5 and (hue < 60 or hue >= 170)):
            return 'red'
        if (sat > 5 and val > 5 and (70 < hue < 90)):
            return 'green'
        else:
            return (hue, sat, val)
    if debug:
        print("Color: ", time.time()-start_time)
    return None
    




In [3]:
# Initialization Stuff
rospy.init_node('liveproc')
listener = tf.TransformListener()
listener.waitForTransform("/odom", "/front_left_camera_link", rospy.Time(), rospy.Duration(2.0))
# pub = rospy.Publisher('chatter', String, queue_size = 100)
occ_map = None
debug = False
camera_range = 75
# robot_pos
# odom_pos

    
np.set_printoptions(threshold=sys.maxsize)

In [4]:
def rectangle(mat,color,image,original, overlay, angle):
    
    global debug
    start_time = time.time()
    rect = cv.boundingRect(mat)
    min_rect = cv.minAreaRect(mat)
    x_bearing = min_rect[0][1]
    height = rect[3]
    width = rect[2]
    x = rect[0]
    x = np.int0(x)
    y = rect[1] 
    y = np.int0(y)
    font = cv.FONT_HERSHEY_SIMPLEX
    tag = 0
    if overlay:
        if (width < height and 4*width > height and width*height > 45):
            #print(color)
#             bearing = ((x+width/2.0)-640.0)/1280.0*80.0
            if(color =='green'):
                tag = 1
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,128,0),2,cv.LINE_AA)
            elif(color =='red'):
                tag = 1
                cv.putText(original,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(0,0,128),2,cv.LINE_AA)
            else:
                tag = 1
                cv.putText(original,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            cv.putText(original,str(x_bearing) + " degrees",(np.int0(x-width/2),np.int0(y-height*math.log(height,10))), font, 0.5,(19,69,139),2,cv.LINE_AA)
            cv.rectangle(original,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = (x_bearing-640.0)/1280.0*80.0
            angle.append(bearing)
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
        return original, angle
    else:
        if (width < height and 4*width > height and width*height > 45):
            #print(color)
#             bearing = ((x+width/2.0)-640.0)/1280.0*80.0
            if(color =='green'):
                tag = 1
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            elif(color =='red'):
                tag = 1
                cv.putText(image,color+' buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            else:
                tag = 1
                cv.putText(image,'buoy',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
            cv.putText(image,str(x_bearing) + " degrees",(np.int0(x-width/2),np.int0(y-height*math.log(height,10))), font, 0.5,(255,255,255),2,cv.LINE_AA)
            cv.rectangle(image,(x,y),(x+width,y+height),(255,0,255),2)
            bearing = ((x_bearing/2.0)-640.0)/1280.0*80.0
            angle.append(bearing)
        if(tag == 0):
            image[y:y+height][x:x+width]=[0,0,0]
            #cv.putText(image,'noise',(np.int0(x-width/2),np.int0(y-height/2.5)), font, 0.5,(255,255,255),2,cv.LINE_AA)
    if debug:
        print("Rectangle: ",time.time()-start_time)
    return image, angle

In [5]:
def createLineIterator(P1, P2, img):
    global debug
    start_time = time.time()
# """
# Produces and array that consists of the coordinates and intensities of
# each pixel in a line between two points

# Parameters:
#     -P1: a numpy array that consists of the coordinate of the first point (x,y)
#     -P2: a numpy array that consists of the coordinate of the second point (x,y)
#     -img: the image being processed

# Returns:
#     -it: a numpy array that consists of the coordinates and intensities of each
#     pixel in the radii (shape: [numPixels, 3], row = [x,y,intensity])     
# """
   #define local variables for readability
    #print img.shape
    image_height = img.shape[0]
    image_width = img.shape[1]
    P1X = P1[0]
    P1Y = P1[1]
    P2X = P2[0]
    P2Y = P2[1]
    
   #difference and absolute difference between points
   #used to calculate slope and relative location between points
    dX = P2X - P1X
    dY = P2Y - P1Y
    dXa = np.abs(dX)
    dYa = np.abs(dY)

    #predefine numpy array for output based on distance between points
    itbuffer = np.empty(shape=(np.maximum(dYa,dXa),3), dtype=np.int32)
    itbuffer.fill(np.nan)

   #Obtain coordinates along the line using a form of Bresenham's algorithm
    negY = P1Y > P2Y
    negX = P1X > P2X
    if P1X == P2X: #vertical line segment
        itbuffer[:,0] = P1X
        if negY:
            itbuffer[:,1] = np.arange(P1Y - 1,P1Y - dYa - 1,-1)
        else:
            itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)              
    elif P1Y == P2Y: #horizontal line segment
        itbuffer[:,1] = P1Y
        if negX:
            itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
        else:
            itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
    else: #diagonal line segment
        steepSlope = dYa > dXa
        if steepSlope:
            slope = dX.astype(np.float32)/dY.astype(np.float32)
            if negY:
                itbuffer[:,1] = np.arange(P1Y-1,P1Y-dYa-1,-1)
            else:
                itbuffer[:,1] = np.arange(P1Y+1,P1Y+dYa+1)
            itbuffer[:,0] = (slope*(itbuffer[:,1]-P1Y)).astype(np.int) + P1X
        else:
            slope = dY.astype(np.float32)/dX.astype(np.float32)
            if negX:
                itbuffer[:,0] = np.arange(P1X-1,P1X-dXa-1,-1)
            else:
                itbuffer[:,0] = np.arange(P1X+1,P1X+dXa+1)
            itbuffer[:,1] = (slope*(itbuffer[:,0]-P1X)).astype(np.int) + P1Y

   #Remove points outside of image
    colX = itbuffer[:,0]
    colY = itbuffer[:,1]
    itbuffer = itbuffer[(colX >= 0) & (colY >=0) & (colX<image_width) & (colY<image_height)]

   #Get intensities from img ndarray
    itbuffer[:,2] = img[itbuffer[:,1].astype(np.uint),itbuffer[:,0].astype(np.uint)]
    if debug:
        print("Lines: ", time.time()-start_time)
    return itbuffer

In [6]:
def build_from_clusters(image, clusters):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    new_img = np.zeros_like(image)
    print_var(clusters)
    for cluster in clusters:
        if isinstance(cluster, np.ndarray):
            sum_x = 0
            sum_y = 0
            for point in cluster:
               sum_x = sum_x + point[1]
               sum_y = sum_y + point[0]
            moment_x = sum_x/(int(cluster.shape[0])-1)
            moment_y = sum_y/(int(cluster.shape[0])-1)
            new_img[moment_x,moment_y] = 255
    return new_img

In [8]:
# For whatever reason, trying to display /anything/ generated by this function
# causes the entire kernel to lag so... look into that
def get_map(p_map, slices, origin):
    global camera_range
    map_array = p_map.data
    map_width = p_map.info.width
    map_height = p_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))

    map_img = np.zeros((map_height, map_width, 3))
#     map_img[map_np == -1] = [255,128,0]
#     map_img[map_np == 0] = [204,0,0]
#     map_img[map_np == 100] = [0,0,0]
    
    map_img[map_np == -1] = [255,145,0]
    map_img[map_np == 0] = [255,145,0]
    map_img[map_np == 100] = [0,154,255]
    
    map_img[origin.y][origin.x] = 
    
    slices_size = len(slices)
#     print(slices_size)
    slices = np.reshape(np.delete(slices, 0, 0), ((len(slices)-1)/3,3))
#     print slices
    time.sleep(100000)
    
#     for i in range(int(slices_size)):
#         try:
#             found = False
#             arr = slices[i]
#             for slice in arr:
#                 print slice
#                 if not found and (slice[2] == -1 or slice[2] == 0):
#                     #map_img[slice[1]][slice[2]]=[102,0,102]
#                     map_img[slice[1]][slice[0]] = [255,255,255]
#                 elif slice[2] == 100:
#                     map_img[slice[1]][slice[0]]=[0,0,255]
#                     found = True
#         except Exception as ex:
#             print ex
#             print slices[i]
#             print
    
    return(map_img)

In [7]:
def map_filter(p_map, trans, robot_yaw, yaw_offset):
    global debug
    start_time = time.time()
    map_array = p_map.data
    origin = p_map.info.origin.position
    
#     odom_po

    map_width = p_map.info.width
    map_height = p_map.info.height
    map_np = np.reshape(np.array(map_array),(map_height,map_width))
    

    print_to_file = False
    robot_pos_x = trans[0]-origin.x
    robot_pos_y = trans[1]-origin.y
    if (robot_yaw < 0):
        robot_yaw = robot_yaw + 360
    i = 0
    f = None
    slices = []
    while (i < len(yaw_offset)):
        buoy_yaw = robot_yaw - yaw_offset[i]
        if ((buoy_yaw) < 0):
            buoy_yaw = buoy_yaw + 360
#         print "robot_yaw", robot_yaw 
#         print "yaw_offset at",i, " : ", yaw_offset[i]
#         print "buoy_yaw= ", buoy_yaw
        global camera_range
        point2_x = camera_range * math.sin(math.radians(buoy_yaw+45))
        point2_y = camera_range * math.cos(math.radians(buoy_yaw+45))
        point1 = np.array([round(robot_pos_x), round(robot_pos_y)])
        point2 = np.array([round(robot_pos_x+point2_x), round(robot_pos_y+point2_y)])
        print "point1", point1
        print "point2", point2
        sliced = createLineIterator(point1.astype(int),point2.astype(int),map_np)
        slices.append(sliced)
        if print_to_file:
            f = open("l.txt", "w+")
            f.write("slice " + str(i) + ": " + str(slices) + "\n")
        i+=1
    if print_to_file:
        f.write("map: " + str(map_np))
        f.write("translation:"+ str(trans))
        f.write("robot yaw:"+ str(robot_yaw))
        f.write("bearing:"+ str(yaw_offset))
        f.close()
#         print robot_yaw, buoy_yaw
#         rospy.sleep(1)
    if debug:
        print("Filter: ", time.time()-start_time)
    return slices

In [9]:
def callback(rect):
    global debug
    global occ_map
    #print "right before"
    start_time = time.time()
    display_rect = False
    angle = []

    rect, bearings = transform(rect,angle)

    if(display_rect):
        cv.namedWindow("stereo/left/image_rect_color",cv.WINDOW_NORMAL)
        cv.imshow("stereo/left/image_rect_color", rect)
        cv.waitKey(1)
    
#     map_image_latest = get_map(p_map)
#     if map_image_latest.size > 0:
#         cv.namedWindow("map",cv.WINDOW_NORMAL)
#         cv.imshow("map", map_image_latest)
#         cv.waitKey(1)
    
    if debug:
        print("Up to listener: ", time.time()-start_time)
    
    global listener 
#     rate = rospy.Rate(0.05)
    if not rospy.is_shutdown():       
        try:
            now = rospy.Time.now()
            listener.waitForTransform("/odom", "/base_link", now, rospy.Duration(2.0))
            (trans,rot) = listener.lookupTransform("/odom", "/base_link", now)
            euler = tf.transformations.euler_from_quaternion(rot)
#             print "robot_yaw= ", math.degrees(euler[2])
            pixel_lines = []
            pixel_lines = map_filter(occ_map, trans, math.degrees(euler[2]), bearings)
#             if pixel_lines != None:
            map_image = get_map(occ_map, pixel_lines, origin)
            cv.namedWindow("map",cv.WINDOW_NORMAL)
            cv.imshow("map", map_image)
            cv.waitKey(1)
        except tf.LookupException as l:
            print l
        except tf.ConnectivityException as c:
            print c
        except tf.ExtrapolationException as e:
            print e
        except Exception as ex:
            print ex
#     global pub
#     string = "hello %s" % rospy.get_time()
#     pub.publish(string)
    timing = False
    if debug or timing:
        print ("Callback: ",time.time()-start_time)

In [10]:
def map_callback(p_map):
    global occ_map
    occ_map = p_map

In [ ]:
if __name__ == '__main__':
    proj_map = rospy.Subscriber("projected_map", Map, map_callback)
    rect = rospy.Subscriber("stereo/left/image_rect_color", Image, callback)
    rospy.spin()
    

'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' 

'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' 

'NoneType' object has no attribute 'write'
robot_yaw 27.6113705208
yaw_offset at 0  :  -34.3092603683
buoy_yaw=  61.9206308892
point1 [ 20. 121.]
point2 [92. 99.]
robot_yaw 27.6858166683
yaw_offset at 0  :  -34.2326593399
buoy_yaw=  61.9184760082
point1 [ 20. 121.]
point2 [92. 99.]
robot_yaw 27.6858166683
yaw_offset at 1  :  -34.2132344246
buoy_yaw=  61.8990510928
point1 [ 20. 121.]
point2 [92. 99.]
robot_yaw 27.7789189397
yaw_offset at 0  :  -34.09375
buoy_yaw=  61.8726689397
point1 [ 21. 121.]
point2 [92. 99.]
'NoneType' object has no attribute 'write'
robot_yaw 27.9111859656
yaw_offset at 0  :  -34.03125
buoy_yaw=  61.9424359656
point1 [ 21. 121.]
point2 [ 93. 100.]
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
robot_yaw 28.0821247622
yaw_offset at 0  :  -34.1926188469
buoy_yaw=  62.2747436091
point1 [ 22. 122.]
point2 [93. 99.]
'NoneType' object has no attribute 'write'
'NoneType' object has no attribute 'write'
'NoneType' object has no attri

robot_yaw 24.0486523153
yaw_offset at 0  :  -34.3257212639
buoy_yaw=  58.3743735792
point1 [ 42. 131.]
point2 [115. 114.]
robot_yaw 24.0486523153
yaw_offset at 1  :  -34.296875
buoy_yaw=  58.3455273153
point1 [ 42. 131.]
point2 [115. 114.]
robot_yaw 19.9017211029
yaw_offset at 0  :  -34.15625
buoy_yaw=  54.0579711029
point1 [ 43. 131.]
point2 [117. 120.]
robot_yaw 19.9017211029
yaw_offset at 1  :  -34.1419110298
buoy_yaw=  54.0436321327
point1 [ 43. 131.]
point2 [117. 120.]
robot_yaw 15.3598198029
yaw_offset at 0  :  -34.2949991226
buoy_yaw=  49.6548189255
point1 [ 43. 132.]
point2 [118. 125.]
robot_yaw 15.3598198029
yaw_offset at 1  :  -34.15625
buoy_yaw=  49.5160698029
point1 [ 43. 132.]
point2 [118. 126.]
robot_yaw 15.3598198029
yaw_offset at 2  :  -34.109375
buoy_yaw=  49.4691948029
point1 [ 43. 132.]
point2 [118. 126.]
robot_yaw 10.4292090607
yaw_offset at 0  :  -34.203125
buoy_yaw=  44.6323340607
point1 [ 44. 132.]
point2 [119. 132.]
robot_yaw 10.4292090607
yaw_offset at 1  :  -3

robot_yaw 15.9209122245
yaw_offset at 0  :  -33.9890160561
buoy_yaw=  49.9099282806
point1 [ 67. 134.]
point2 [142. 127.]
robot_yaw 15.9209122245
yaw_offset at 1  :  -33.71875
buoy_yaw=  49.6396622245
point1 [ 67. 134.]
point2 [142. 128.]
robot_yaw 19.6728119052
yaw_offset at 0  :  -34.125
buoy_yaw=  53.7978119052
point1 [ 67. 134.]
point2 [141. 122.]
robot_yaw 19.6728119052
yaw_offset at 1  :  -34.125
buoy_yaw=  53.7978119052
point1 [ 67. 134.]
point2 [141. 122.]
robot_yaw 19.6728119052
yaw_offset at 2  :  -33.8053832054
buoy_yaw=  53.4781951106
point1 [ 67. 134.]
point2 [141. 123.]
robot_yaw 19.6728119052
yaw_offset at 3  :  -33.84375
buoy_yaw=  53.5165619052
point1 [ 67. 134.]
point2 [141. 123.]
robot_yaw 22.989298026
yaw_offset at 0  :  -34.234375
buoy_yaw=  57.223673026
point1 [ 67. 134.]
point2 [141. 118.]
robot_yaw 22.989298026
yaw_offset at 1  :  -33.96875
buoy_yaw=  56.958048026
point1 [ 67. 134.]
point2 [141. 118.]
robot_yaw 22.989298026
yaw_offset at 2  :  -34.21875
buoy_yaw

...

robot_yaw 9.34244089059
yaw_offset at 0  :  -34.671875
buoy_yaw=  44.0143158906
point1 [ 85. 140.]
point2 [160. 141.]
robot_yaw 9.34244089059
yaw_offset at 1  :  -34.625
buoy_yaw=  43.9674408906
point1 [ 85. 140.]
point2 [160. 141.]
robot_yaw 9.34244089059
yaw_offset at 2  :  -33.6414418221
buoy_yaw=  42.9838827126
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34244089059
yaw_offset at 3  :  -33.578125
buoy_yaw=  42.9205658906
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34325647949
yaw_offset at 0  :  -34.390625
buoy_yaw=  43.7338814795
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34325647949
yaw_offset at 1  :  -34.34375
buoy_yaw=  43.6870064795
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34325647949
yaw_offset at 2  :  -33.317065239
buoy_yaw=  42.6603217184
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34325647949
yaw_offset at 3  :  -33.28125
buoy_yaw=  42.6245064795
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34400800532
yaw_offset at 0  :  -34.074874

robot_yaw 9.34285005455
yaw_offset at 0  :  -34.2317576408
buoy_yaw=  43.5746076954
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34285005455
yaw_offset at 1  :  -34.1875
buoy_yaw=  43.5303500546
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34285005455
yaw_offset at 2  :  -33.1912727356
buoy_yaw=  42.5341227901
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34285005455
yaw_offset at 3  :  -33.1875
buoy_yaw=  42.5303500546
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34222356285
yaw_offset at 0  :  -34.34375
buoy_yaw=  43.6859735629
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34222356285
yaw_offset at 1  :  -34.296875
buoy_yaw=  43.6390985629
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34222356285
yaw_offset at 2  :  -33.3190536499
buoy_yaw=  42.6612772128
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34222356285
yaw_offset at 3  :  -33.28125
buoy_yaw=  42.6234735629
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34122816813
yaw_offset at 0  :  -34.47

robot_yaw 9.34537685685
yaw_offset at 0  :  -34.171875
buoy_yaw=  43.5172518568
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34537685685
yaw_offset at 1  :  -34.125
buoy_yaw=  43.4703768568
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34537685685
yaw_offset at 2  :  -33.0995707512
buoy_yaw=  42.444947608
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34537685685
yaw_offset at 3  :  -33.0625
buoy_yaw=  42.4078768568
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34605316262
yaw_offset at 0  :  -33.921875
buoy_yaw=  43.2679281626
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34605316262
yaw_offset at 1  :  -33.875
buoy_yaw=  43.2210531626
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34605316262
yaw_offset at 2  :  -32.875
buoy_yaw=  42.2210531626
point1 [ 85. 140.]
point2 [160. 144.]
robot_yaw 9.34605316262
yaw_offset at 3  :  -32.859375
buoy_yaw=  42.2054281626
point1 [ 85. 140.]
point2 [160. 144.]
robot_yaw 9.34605316262
yaw_offset at 4  :  -31.671875
buoy_yaw=

robot_yaw 9.34160154819
yaw_offset at 0  :  -34.53125
buoy_yaw=  43.8728515482
point1 [ 85. 140.]
point2 [160. 141.]
robot_yaw 9.34160154819
yaw_offset at 1  :  -34.484375
buoy_yaw=  43.8259765482
point1 [ 85. 140.]
point2 [160. 141.]
robot_yaw 9.34160154819
yaw_offset at 2  :  -33.5608115196
buoy_yaw=  42.9024130678
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34160154819
yaw_offset at 3  :  -33.390625
buoy_yaw=  42.7322265482
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34160154819
yaw_offset at 4  :  -32.0144238472
buoy_yaw=  41.3560253954
point1 [ 85. 140.]
point2 [160. 145.]
robot_yaw 9.34053681901
yaw_offset at 0  :  -34.5352888107
buoy_yaw=  43.8758256297
point1 [ 85. 140.]
point2 [160. 141.]
robot_yaw 9.34053681901
yaw_offset at 1  :  -34.484375
buoy_yaw=  43.824911819
point1 [ 85. 140.]
point2 [160. 141.]
robot_yaw 9.34053681901
yaw_offset at 2  :  -33.6265907288
buoy_yaw=  42.9671275478
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34053681901
yaw_offset at 3  :

robot_yaw 9.34504289955
yaw_offset at 0  :  -33.8125
buoy_yaw=  43.1575428995
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34504289955
yaw_offset at 1  :  -33.765625
buoy_yaw=  43.1106678995
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34504289955
yaw_offset at 2  :  -32.8304243088
buoy_yaw=  42.1754672083
point1 [ 85. 140.]
point2 [160. 144.]
robot_yaw 9.34504289955
yaw_offset at 3  :  -32.8059163094
buoy_yaw=  42.1509592089
point1 [ 85. 140.]
point2 [160. 144.]
robot_yaw 9.34446871581
yaw_offset at 0  :  -33.90625
buoy_yaw=  43.2507187158
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34446871581
yaw_offset at 1  :  -33.859375
buoy_yaw=  43.2038437158
point1 [ 85. 140.]
point2 [160. 142.]
robot_yaw 9.34446871581
yaw_offset at 2  :  -32.9297261238
buoy_yaw=  42.2741948396
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.34446871581
yaw_offset at 3  :  -32.924161911
buoy_yaw=  42.2686306268
point1 [ 85. 140.]
point2 [160. 143.]
robot_yaw 9.3436591091
yaw_offset at 0  :  -